In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
import numpy as np

def f1(x: np.ndarray, t: np.ndarray) -> np.ndarray:
    return 1.0 / np.cosh(x + 3.0) * np.exp(2.3J * t)

def f2(x: np.ndarray, t: np.ndarray) -> np.ndarray:
    return 2.0 / np.cosh(x) * np.tanh(x) * np.exp(2.8J * t)

x = np.linspace(-5.0, 5.0, 2 ** 10 + 1)
t = np.linspace(0.0, 4.0 * np.pi, 2 ** 8 + 1)

xgrid, tgrid = np.meshgrid(x, t)

X1 = f1(xgrid, tgrid)
X2 = f2(xgrid, tgrid)
X = X1 + X2

In [ ]:
from matplotlib import pyplot as plt

titles = ['$f_1(x,t)$', '$f_2(x,t)$', '$f$']
data = [X1, X2, X]

fig = plt.figure(figsize=(16, 9), dpi=400)
for pos, title, c in zip(range(131, 134), titles, data):
    plt.subplot(pos, facecolor='white', title=title)
    plt.pcolor(xgrid, tgrid, c.real)
plt.colorbar()
plt.show()

In [ ]:
from numpy.linalg import pinv

EPSILON = 1E-8
kwargs = {'atol': EPSILON, 'rtol': EPSILON}

n = len(x)
A = X[1:].T @ pinv(X[:-1].T, rcond=EPSILON)
assert A.shape == (n, n)
assert np.allclose(A @ X[:-1].T, X[1:].T, **kwargs)

In [ ]:
from numpy.linalg import eig, lstsq, svd

class DynamicModeDecomposition:
    def __init__(self, svd_rank, exact: bool = False):
        if (not isinstance(svd_rank, (float, int))
                or not isinstance(exact, bool)):
            raise ValueError()
        self.svd_rank, self.exact = svd_rank, exact

    def fit(self, x: np.ndarray):
        if not isinstance(x, np.ndarray):
            raise TypeError()
        if len(x.shape) != 2 or len(x) >= x.shape[1] or len(x) < 2:
            raise ValueError()
        x, x_prime = x[:-1].T, x[1:].T
        u, s, vh = svd(x, full_matrices=False)
        assert np.allclose(x, u * s @ vh, **kwargs)
        u, s, vh = u[:, :self.svd_rank], s[:self.svd_rank], vh[:self.svd_rank]
        assert x.shape == (u * s @ vh).shape
        self.atilde = np.conj(u.T) @ x_prime @ np.conj(vh.T) / s
        self.eigs, v = eig(self.atilde)
        assert np.allclose(self.atilde @ v, self.eigs * v, **kwargs)
        if self.exact:
            self.modes = x_prime @ np.conj(vh.T) / s @ v
        else:
            self.modes = u @ v
        b = lstsq(self.modes, x[:, 0])
        return self

    def predict(self, x: np.array):
        return x

In [ ]:
from pydmd import DMD

for exact in [False, True]:
    dmd = DMD(svd_rank=2, exact=exact).fit(X.T)
    dynamic_mode_decomposition = DynamicModeDecomposition(2, exact=exact).fit(X)
    assert np.allclose(dynamic_mode_decomposition.atilde, dmd.atilde, **kwargs)
    assert np.allclose(dynamic_mode_decomposition.eigs, dmd.eigs, **kwargs)
    assert np.allclose(dynamic_mode_decomposition.modes, dmd.modes, **kwargs)
    assert np.allclose(dmd.predict(X[:-1].T), X[1:].T, **kwargs)